# Analyzing Steam's 2024 Data

This project aims to explore Steam’s 2024 data with a focus on total sales, publisher performance, and game distribution. The workbook will showcase data cleaning, SQL querying, and data visualization using Tableau Public to uncover the data and information.

***

# 1. Introduction

Steam is the largest platform for PC gaming, and analyzing its data offers valuable insights into game sales, publisher performance, and market trends. This project aims to explore:

* Steam Statistics of 2024
* Copies sold across these games
* Game reviews, revenue and average playtimes
* Publisher data, including:
 * Revenue per game publisher
 * Number of games published, categorized by publisher class.

# Objectives
* Use SQL to clean and prepare the dataset
* Query to explore data and find specific insights
* Visualize the data through Tableau Public for meaningful storytelling

***

# 2. Data Overview

The dataset consists of Steam game statas for 2024. It contains the following columns:

* name: The name of the game
* releaseDate: The release date of the game
* copiesSold: Total copies sold
* price: The price of the game
* revenue: Total revenue from the game
* avgPlaytime: Average playtime per player
* reviewScore: User rating score of the game
* publisherClass: Classification of publishers (Indie, AAA, etc.)
* publishers: Names of the game publishers
* developers: Names of the game developers
* steamId: Unique identifier for each game on Steam

***

# 3. Query and Documentation
# Initial imports and connecting BigQuery to Kaggle


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/top-1500-games-on-steam-by-revenue-09-09-2024/Steam_2024_bestRevenue_1500.csv


In [6]:
# Import BigQuery to Kaggle
from google.cloud import bigquery
bigquery_client = bigquery.Client(project='steam-435621', location='US')

# Import the BQ API Client library
from google.cloud import bigquery
client = bigquery.Client(project='steam-435621', location='US')

In [7]:
# Construct a reference to the steam dataset that is within the project
dataset_ref = client.dataset('steam_data', project='steam-435621')

# Make an API request to fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [8]:
# Make a list of all the tables in the dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset
for table in tables:  
    print(table.table_id)

steam_db


# Previewing the data

In [9]:
# Preview the first ten lines of the table
client.list_rows(table, max_results=10).to_dataframe()

,name,releaseDate,copiesSold,price,revenue,avgPlaytime,reviewScore,publisherClass,publishers,developers,steamId
0,Spectre Divide,2024-09-03,584912.0,0.0,515874.00,3.79,53,Indie,Mountaintop Studios,Mountaintop Studios,2641470
1,Lust Goddess,2024-02-21,456707.0,0.0,9695496.00,33.54,48,Indie,BRAWEA LTD,BRAWEA LTD,2808930
2,Stormgate,2024-07-30,224772.0,0.0,1459213.00,7.34,52,Indie,Frost Giant Studios,Frost Giant Studios,2012510
3,ASTRA: Knights of Veda,2024-04-01,168093.0,0.0,2460116.00,50.12,42,Indie,HYBE IM,FLINT,2484250
4,Atelier Resleriana: Forgotten Alchemy and the ...,2024-01-24,137669.0,0.0,2197863.00,37.11,53,Indie,KOEI TECMO GAMES,"KOEI TECMO GAMES,Akatsuki Games Inc.",2594920
5,失乐星图Nornium,2024-08-09,67552.0,0.0,60846.00,2.18,15,Indie,Flaring Cloister,Flaring Cloister,2877160
6,BATTLE CRUSH,2024-06-26,62503.0,0.0,49147.00,16.56,49,Indie,NCSOFT Corporation,NCSOFT Corporation,2287920
7,ASTRAL TALE-星界神話,2024-01-07,43217.0,0.0,138164.27,104.18,39,Indie,"X-Legend Entertainment Co., Ltd.","X-Legend Entertainment Co., Ltd.",2591210
8,Growtopia,2024-03-07,37131.0,0.0,65173.00,7.36,31,AAA,Ubisoft,Ubisoft Abu Dhabi,866020
9,Rocksmith+,2024-06-06,35660.0,0.0,193594.00,3.87,11,AAA,Ubisoft,"Ubisoft San Francisco,Ubisoft Osaka,Ubisoft Bu...",2834910


# Data cleaning

In [10]:
# Make sure all the data are loaded

query1 = """
          SELECT 
            *
          FROM 
            `steam-435621.steam_data.steam_db`          
        """

# Set up the query
query_job1 = client.query(query1)

# Make an API request  to run the query and return a pandas DataFrame
housestyleAC = query_job1.to_dataframe()

# See the resulting table made from the query
print(housestyleAC)

                                                   name releaseDate  \
0                                        Spectre Divide  2024-09-03   
1                                          Lust Goddess  2024-02-21   
2                                             Stormgate  2024-07-30   
3                                ASTRA: Knights of Veda  2024-04-01   
4     Atelier Resleriana: Forgotten Alchemy and the ...  2024-01-24   
...                                                 ...         ...   
1495                               Our Adventurer Guild  2024-04-12   
1496                                  TRAMCITY HAKODATE  2024-01-04   
1497                                 DEVIL BLADE REBOOT  2024-05-23   
1498                    Dünnes Eis - Das Spiel zum Song  2024-05-16   
1499                                    Mirage Feathers  2024-09-06   

      copiesSold  price    revenue  avgPlaytime  reviewScore publisherClass  \
0       584912.0   0.00   515874.0         3.79           53        

In [11]:
# Check for any missing data
query2 = """
          SELECT 
            COUNT(*) AS total_rows,
            COUNT(name) AS name_non_null,
            COUNT(revenue) AS revenue_non_null
          FROM 
            `steam-435621.steam_data.steam_db`          
        """

# Set up the query
query_job2 = client.query(query2)

# Make an API request  to run the query and return a pandas DataFrame
housestyleAC = query_job2.to_dataframe()

# See the resulting table made from the query
print(housestyleAC)

   total_rows  name_non_null  revenue_non_null
0        1500           1500              1500


# Data Exploration

In [12]:
# What are the top 15 steam games of 2024 based on revenue
# Converted the revenue to $ format, 2 decimal places with (,) as separators

# Write the query
query3 = """
          SELECT 
            name,
          CONCAT
            ('$', FORMAT("%'.2f", revenue)) AS formatted_revenue
          FROM 
            `steam-435621.steam_data.steam_db`          
          ORDER BY revenue
          DESC
          LIMIT 15;
        """

# Set up the query
query_job3 = client.query(query3)

# Make an API request  to run the query and return a pandas DataFrame
housestyleAC = query_job3.to_dataframe()

# See the resulting table made from the query
print(housestyleAC)

                                name formatted_revenue
0                 Black Myth: Wukong   $837,793,356.00
1                      HELLDIVERS™ 2   $435,635,596.00
2                           Palworld   $392,328,553.00
3                 Sons Of The Forest   $217,017,892.00
4                   Dragon's Dogma 2   $111,478,291.00
5               The First Descendant   $102,244,808.00
6                         Last Epoch    $97,723,674.00
7                      7 Days to Die    $89,781,931.00
8                           V Rising    $83,614,738.00
9                        Manor Lords    $63,098,408.00
10                        Enshrouded    $57,595,515.00
11  Ghost of Tsushima DIRECTOR'S CUT    $49,948,315.00
12          Granblue Fantasy: Relink    $49,035,237.00
13                          TEKKEN 8    $45,285,110.00
14                          Hades II    $42,718,547.00


In [13]:
# Load data of paid games that sold more than 1M copies and copies sold respectively
query4 = """
          SELECT 
            name,
          CONCAT
          ('$',FORMAT("%'.2f", price)) AS formatted_price,
          FORMAT
          ("%'.2f", copiesSold) AS formatted_copiesSold,
          CONCAT
          ('$',FORMAT("%'.2f", revenue)) AS formatted_revenue
          FROM
          `steam-435621.steam_data.steam_db`
          WHERE
          copiesSold > 1000000
          AND
          price > 0
          ORDER BY
          copiesSold
          DESC;
        """

# Set up the query
query_job4 = client.query(query4)

# Make an API request  to run the query and return a pandas DataFrame
housestyleAC = query_job4.to_dataframe()

# See the resulting table made from the query
print(housestyleAC)

                            name formatted_price formatted_copiesSold  \
0                       Palworld          $29.99        16,704,850.00   
1             Black Myth: Wukong          $59.99        15,517,278.00   
2                  HELLDIVERS™ 2          $39.99        11,905,198.00   
3                  7 Days to Die          $44.99         9,877,443.00   
4             Sons Of The Forest          $29.99         8,693,478.00   
5                       V Rising          $34.99         4,784,609.00   
6               Chained Together           $4.99         3,946,801.00   
7                     Last Epoch          $34.99         3,300,623.00   
8          Supermarket Simulator          $12.99         2,514,219.00   
9                     Enshrouded          $29.99         2,338,262.00   
10                   Manor Lords          $39.99         2,294,915.00   
11                   Core Keeper          $19.99         2,023,964.00   
12              Dragon's Dogma 2          $69.99   

In [19]:
# Load data of paid games that sold more than 1M copies and copies sold respectively
query6 = """
          SELECT 
            publishers,
          SUM
            (revenue) AS total_revenue,
          AVG
            (reviewScore) AS avg_review_score,
          COUNT
            (name) AS total_games
          FROM
            `steam-435621.steam_data.steam_db`
          GROUP BY 
            publishers
          ORDER BY 
            total_revenue
          DESC
          LIMIT
            30;
        """

# Set up the query
query_job6 = client.query(query6)

# Make an API request  to run the query and return a pandas DataFrame
housestyleAC = query_job6.to_dataframe()

# See the resulting table made from the query
print(housestyleAC)

                         publishers  total_revenue  avg_review_score  \
0                      Game Science   8.377934e+08         96.000000   
1        PlayStation Publishing LLC   5.088852e+08         81.250000   
2                        Pocketpair   3.923286e+08         94.000000   
3                          Newnight   2.170179e+08         86.000000   
4                  CAPCOM Co., Ltd.   1.178810e+08         74.200000   
5                             NEXON   1.022448e+08         55.000000   
6               Eleventh Hour Games   9.772367e+07         86.000000   
7   The Fun Pimps Entertainment LLC   8.978193e+07         89.000000   
8                  Stunlock Studios   8.361474e+07          0.000000   
9                      Hooded Horse   7.848720e+07         83.800000   
10                             SEGA   6.352835e+07         94.750000   
11                  Keen Games GmbH   5.759552e+07         87.000000   
12                    Cygames, Inc.   4.971079e+07         78.00

In [14]:
# Load final dataset to use for analysis
query5 = """
          SELECT 
            name,
          FORMAT
            ("%'.2f", price) AS formatted_price,
          FORMAT
            ("%'.2f", copiesSold) AS formatted_copiesSold,
          FORMAT
            ("%'.2f", revenue) AS formatted_revenue,
          FORMAT
            ("%'.2f", avgPlaytime) AS formatted_avgPlaytime,
            reviewScore, publisherClass, publishers,
          FROM
          `steam-435621.steam_data.steam_db`
          ORDER BY
          revenue
          DESC;
        """

# Set up the query
query_job5 = client.query(query5)

# Make an API request  to run the query and return a pandas DataFrame
housestyleAC = query_job5.to_dataframe()

# See the resulting table made from the query
print(housestyleAC)

                                              name formatted_price  \
0                               Black Myth: Wukong           59.99   
1                                    HELLDIVERS™ 2           39.99   
2                                         Palworld           29.99   
3                               Sons Of The Forest           29.99   
4                                 Dragon's Dogma 2           69.99   
...                                            ...             ...   
1495                                Champion Shift            6.99   
1496             Megacopter: Blades of the Goddess           15.99   
1497  DYSCHRONIA: Chronos Alternate - Dual Edition           34.99   
1498                              Claw Machine Sim            6.99   
1499                 Memories Off #5 Togireta Film           14.99   

     formatted_copiesSold formatted_revenue formatted_avgPlaytime  \
0           15,517,278.00    837,793,356.00                 20.07   
1           11,905,19

***

# 4. Data Visualization using Tableau Public

# Data story via Tableau Public

After querying, I used Tableau Public to create the following visualizations:

* Games Sold vs Revenue: A bar chart that displays total revenue by games, paired with the amount of games sold.
* Game Price vs Games Sold: A scatter plot showing the relationship between the number of copies sold and the price of the game, revealing trends between high-priced games and copies sold.

* Publisher Class Distribution: A simple pie chart visualizing the publisher class distribution by AAA, AA, Indie studios and Hobbyists.
* Total Revenue by Publisher: A line chart showing the revenue earned publishing companies for 2024.

* Review Score vs. Playtime: A bubble chart comparing average review scores with playtime, revealing how player satisfaction correlates with engagement.

In [22]:
%%HTML
<div class='tableauPlaceholder' id='viz1727102281015' style='position: relative'><noscript><a href='#'><img alt='2024 Steam Statistics ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Q2&#47;Q25K6CTG8&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='shared&#47;Q25K6CTG8' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Q2&#47;Q25K6CTG8&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1727102281015');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='1016px';vizElement.style.height='991px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

# Data Dashboard

This dashboard provides an in-depth analysis of Steam game sales for 2024, focusing on key metrics providing a comprehensive view of market trends in the gaming industry.

In [21]:
%%HTML
<div class='tableauPlaceholder' id='viz1727101817403' style='position: relative'><noscript><a href='#'><img alt='Dashboard 2 ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;St&#47;SteamData_17271014029670&#47;Dashboard2&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='SteamData_17271014029670&#47;Dashboard2' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;St&#47;SteamData_17271014029670&#47;Dashboard2&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1727101817403');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='1366px';vizElement.style.height='795px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='1366px';vizElement.style.height='795px';} else { vizElement.style.width='100%';vizElement.style.height='1877px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

# 5. Acknowledgements

Ali Cem Topcu. (2024 September). Top 1500 games on steam by revenue 09-09-2024, version 1. Retrieved 2024 September 10 from https://www.kaggle.com/datasets/alicemtopcu/top-1500-games-on-steam-by-revenue-09-09-2024/data